# Detecting the rectangle of the rice leaf
## This pybook is first of the development process

In [2]:
import tensorflow as tf
import numpy as np


In [3]:
STD_SIZE: int = 1600
def load_image(filepath: str):
    file_content = tf.io.read_file(filepath)
    image = tf.io.decode_jpeg(file_content, channels=3) # load in RGB
    return tf.image.resize(image, [STD_SIZE, STD_SIZE]) # don't forget to uniform our image

In [4]:
import PIL
image = load_image("RiceLeafs/train/Healthy/IMG_20190419_123623.jpg")
image = tf.cast(image, tf.float32)
image = tf.expand_dims(image, 0)

sobel = tf.image.sobel_edges(image)
sobel_y = np.asarray(sobel[0, :, :, :, 0]) # sobel in y-direction
sobel_x = np.asarray(sobel[0, :, :, :, 1]) # sobel in x-direction

PIL.Image.fromarray(sobel_y[..., 0]  ).show()
PIL.Image.fromarray(sobel_x[..., 0]  ).show()

2022-01-16 20:44:45.393130: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-16 20:44:45.394504: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
import cv2
#reading the image
image = cv2.imread("RiceLeafs/train/Healthy/IMG_20190419_123623.jpg")
image = cv2.resize(image, (1600, 1600))
edged = cv2.Canny(image, 10, 220)
#cv2.imshow("Edges", edged)
#cv2.waitKey(0)

#applying closing function
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
#cv2.imshow("Closed", closed)
#cv2.waitKey(0)

#finding_contours
(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
def find_longest_dicks(contours) -> list: # distance in contours klist
    full_list = []
    for c in contours:
        peri = cv2.arcLength(c, True)
        poly = cv2.approxPolyDP(c, 0.02 * peri, True)
        for lin_i in range(len(poly))[:-1]:
            print(poly[lin_i][0])
            #full_list.append((poly[lin_i], poly[lin_i+1]))
            full_list.append(((poly[lin_i][0][0], poly[lin_i][0][1]), (poly[lin_i+1][0][0], poly[lin_i+1][0][1])))
    full_list = sorted(full_list, key=lambda x: -(x[0][0] - x[1][0])**2 - (x[0][1] - x[1][1])**2)
    return full_list


In [60]:
import cv2
from scipy import ndimage

scharr = np.array([[1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1]])

sharpen = np.array([[0, 0, 0, -1, 0, 0, 0],
                    [0, 0, -1, -1, -1, 0, 0],
                    [0, -1, -1, -1, -1, -1, 0],
                    [-1, -1, -1, 24, -1, -1, -1],
                    [0, -1, -1, -1, -1, -1, 0],
                    [0, 0, -1, -1, -1, 0, 0],
                    [0, 0, 0, -1, 0, 0, 0]])

new_conv = np.array([[0, -1, 0],
                     [-1, 5, -1],
                     [0, -1, 0]])

image = cv2.imread("RiceLeafs/train/Healthy/IMG_20190419_123623.jpg")
image = cv2.resize(image, (1600, 1600))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=21)
grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=21)

abs_grad_x = cv2.convertScaleAbs(grad_x)
abs_grad_y = cv2.convertScaleAbs(grad_y)

grad = np.sqrt(grad_x**2 + grad_y**2)
grad_norm = (grad * 255 / grad.max()).astype(np.uint8)
grad_zero = grad_norm.copy()
grad_zero[np.abs(grad_zero) < 64] = 0
grad_zero = grad_zero.astype(np.float)
grad_cnn = ndimage.convolve(grad_zero, sharpen, mode='constant', cval=0.0)
grad_cnn[grad_cnn > 255] = 255
grad_cnn[grad_cnn < 64] = 0
grad_cnn = grad_cnn.astype(np.uint8)
print(grad_cnn.min(), grad_cnn.max())


kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(grad_cnn, cv2.MORPH_CLOSE, kernel)

(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    #print(approx, len(approx))
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)

longest = find_longest_dicks(cnts)
print(longest)
for line in longest[:2]:
    cv2.line(image, line[0], line[1], color=(255, 0, 0), thickness=2)

#grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

#PIL.Image.fromarray(grad_x  ).show()
#PIL.Image.fromarray(grad_y  ).show()
#PIL.Image.fromarray(grad_norm  ).show()
#PIL.Image.fromarray(closed  ).show()
PIL.Image.fromarray(image  ).show()
#PIL.Image.fromarray(grad_zero  ).show()
PIL.Image.fromarray(np.absolute(grad_cnn)  ).show()
#cv2.imshow('grad X',grad_x)
#cv2.imshow('grad Y',grad_y)
#cv2.imshow('Sobel Image',grad)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

/tmp/ipykernel_9643/465176974.py:38: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grad_zero = grad_zero.astype(np.float)


0 255
[1549 1160]
[1576 1174]
[1586 1176]
[1579 1174]
[1049  854]
[1045  859]
[1035  858]
[1039  862]
[1044  863]
[628 769]
[631 771]
[634 771]
[640 775]
[642 775]
[641 773]
[637 771]
[635 771]
[613 765]
[785 760]
[788 763]
[792 763]
[804 769]
[572 744]
[574 746]
[575 746]
[576 747]
[577 747]
[577 746]
[576 745]
[575 745]
[535 730]
[555 739]
[558 739]
[583 728]
[581 728]
[580 729]
[578 729]
[578 730]
[581 732]
[582 731]
[529 727]
[531 729]
[533 729]
[520 722]
[520 723]
[523 726]
[524 726]
[524 725]
[490 712]
[494 715]
[503 717]
[508 720]
[513 720]
[467 700]
[469 700]
[475 706]
[484 708]
[480 704]
[476 704]
[460 698]
[461 698]
[462 699]
[463 699]
[462 699]
[445 691]
[445 692]
[448 694]
[452 694]
[454 695]
[453 693]
[451 692]
[448 692]
[418 681]
[420 683]
[421 683]
[422 684]
[424 684]
[424 683]
[413 678]
[414 679]
[416 679]
[404 674]
[406 676]
[407 676]
[408 677]
[410 677]
[410 676]
[409 675]
[408 675]
[334 659]
[335 660]
[359 656]
[395 671]
[320 654]
[320 655]
[322 655]
[323 656]
[326 6